In [1]:
from azure.cosmos import CosmosClient, exceptions

import csv 
import json
import sys

%load_ext autoreload
%autoreload
from src.AzureCosmos_connector import COSMOS_CONNECTOR

In [2]:
cosmosDB = COSMOS_CONNECTOR()

In [3]:
cosmosDB.list_databases()
print("="*10)
cosmosDB.find_database("image-embeddings-db")
print("="*10)
emd_db = cosmosDB.get_database("image-embeddings-db")

Databases:
image-embeddings-db
Database with id 'image-embeddings-db' was found
Database with id 'image-embeddings-db' was found, it's link is dbs/image-embeddings-db


In [4]:
cosmosDB.list_containers(emd_db)
print("="*10)
cosmosDB.find_container(emd_db, "clustered-meta-data")
print("="*10)
metaContainer= cosmosDB.get_container(emd_db, "clustered-meta-data")

Containers:
clustered-meta-data
bert-encodings
Container with id 'clustered-meta-data' was found
Container with id 'clustered-meta-data' was found, it's link is dbs/image-embeddings-db/colls/clustered-meta-data


In [ ]:
""" will get a large datafram (1GB)
use by cautious
"""
cosmosDB.get_full_document_to_pd(metaContainer, if_pandas=True).head(2)

In [5]:
ids = ["images_000000000001", "images_000000000019", "images_000000000063"]
meta_data = cosmosDB.query_by_multiId(metaContainer, ids, if_pandas=True)
meta_data

,id,cluster_id,cluster_centroid,dataset,url,encd,img_name,_rid,_self,_etag,_attachments,_ts
0,images_000000000001,9,"[2.1947617275457807, 2.295159087601433, 5.0072...",images,https://cs5425imagedata.blob.core.windows.net/...,"[-0.39324715733528137, 0.440214067697525, -1.1...",000000000001,01krAORYUVkBAAAAAAAAAA==,dbs/01krAA==/colls/01krAORYUVk=/docs/01krAORYU...,"""45007e1a-0000-1900-0000-62476a470000""",attachments/,1648847431
1,images_000000000019,24,"[0.8237275222864728, 11.10838134006322, -3.727...",images,https://cs5425imagedata.blob.core.windows.net/...,"[-0.007508492562919855, 0.4850575923919678, -0...",000000000019,01krAORYUVkDAAAAAAAAAA==,dbs/01krAA==/colls/01krAORYUVk=/docs/01krAORYU...,"""4500801a-0000-1900-0000-62476a480000""",attachments/,1648847432
2,images_000000000063,1,"[0.6532268352227274, -1.5335694497908743, -0.5...",images,https://cs5425imagedata.blob.core.windows.net/...,"[0.09131952375173569, -0.8119602203369141, -0....",000000000063,01krAORYUVkFAAAAAAAAAA==,dbs/01krAA==/colls/01krAORYUVk=/docs/01krAORYU...,"""4500821a-0000-1900-0000-62476a480000""",attachments/,1648847432


In [13]:
query_embedding = meta_data["encd"].iloc[1]
print(query_embedding[:10])
print(meta_data["id"].iloc[1])

[-0.007508492562919855, 0.4850575923919678, -0.656403124332428, -0.37877821922302246, -0.2858748137950897, 0.5738587975502014, -0.40583816170692444, 0.2839474678039551, -0.09739869087934494, -0.49087604880332947]
images_000000000019


In [8]:
%load_ext autoreload
%autoreload
from src.topKImages import topKImages_Azure
image_res, similarities = topKImages_Azure(query_embedding, k=20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Database with id 'image-embeddings-db' was found, it's link is dbs/image-embeddings-db
Container with id 'clustered-meta-data' was found, it's link is dbs/image-embeddings-db/colls/clustered-meta-data
Query belongs to cluster 24


In [9]:
similarities

array([0.87217975, 0.87964899, 0.88289172, 0.88691453, 0.89066712,
       0.8921121 , 0.89522343, 0.89689387, 0.89705573, 0.89989139,
       0.89991912, 0.90408749, 0.91310273, 0.9207164 , 0.92126763,
       0.92398308, 0.92481768, 0.93271062, 0.96702883, 1.        ])

In [10]:
image_res

array(['000000087167', '4ff7332ca734b0ba', '4767621f956b3d0c',
       '000000097442', '000000181225', 'd48a0f2253202d99',
       '58b3906f2adfe7a1', '000000026689', '000000139925',
       'd95cb72e629f07dc', 'a4e8f96810dab5f3', '650a5da733ff17fe',
       '000000309691', '000000193437', '000000012127', '000000246325',
       '224ad16181d9fb6b', '000000200581', 'a220a5426b21e90a',
       '000000000019'], dtype='<U16')